**Keywords**: Pagerank, Power Method

**About the dataset**: \
*DBpedia* (from "DB" for "database") is a project aiming to extract structured content from the information created in the Wikipedia project. This structured information is made available on the World Wide Web. DBpedia allows users to semantically query relationships and properties of Wikipedia resources, including links to other related datasets. for more info, see: https://en.wikipedia.org/wiki/DBpedia. \
We will download two files from the data respository:
* The first file -- **redirects_en.nt.bz2** -- contains redirects link for a page. Let A redirect to B and B redirect to C. Then we will replace article A by article C wherever needed.
* The second file -- **page_links_en.nt.bz2** -- contains pagelinks which are links within an article to other wiki article.

Note that the data is both files is a list of lines which can be split into 4 parts:
* The link to first article.
* Whether it is a redirect, or a pagelink.
* The link to second article.
* A fullstop.

#### <font color="red">Note:</font> Any line which cannot be split into 4 parts is skipped from consideration.

**Agenda**:
* In this notebook, I will be implementing the [*google pagerank algorithm*](https://towardsdatascience.com/pagerank-algorithm-fully-explained-dc794184b4af) to determine the most important articles.
* This will be done by computing the principal eigenvector of the article-article graph adjacency matrix.
* I will be applying the *power method* to extract the principal eigenvector from the adjacency matrix. 
* Using the computed eigenvector, we can assign each article a eigenvector-centrality score. Then we can determine the most important articles.

**Environment**:
Ensure following libraries are installed
- sklearn
- numpy

Also ensure that you have around **4 GB** of memory.



---



In [1]:
# imports
import pickle
from bz2 import BZ2File
import bz2
import os
from datetime import datetime
import pprint
from time import time
import numpy as np
from urllib.request import urlopen
import scipy.sparse as sparse
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# downloading the dataset and store files in local

# dbpedia download urls
redirects_url = "http://downloads.dbpedia.org/3.5.1/en/redirects_en.nt.bz2"
page_links_url = "http://downloads.dbpedia.org/3.5.1/en/page_links_en.nt.bz2"

# extracting the file-names from the urls. Needed to load the files later
redirects_filename = redirects_url.rsplit("/", 1)[1] # redirects_en.nt.bz2 ~ 59MB
page_links_filename = page_links_url.rsplit("/", 1)[1] # page_links_en.nt.bz2 ~ 850MB

resources = [
    (redirects_url, redirects_filename),
    (page_links_url, page_links_filename),
]

# downloading the files
# this will take some time
for url, filename in resources:
    if not os.path.exists(filename):
        print("Downloading data from '%s', please wait..." % url)
        opener = urlopen(url)
        open(filename, "wb").write(opener.read())
        print()

In [3]:
# loading the data from the downloaded files

#reading redirects file
redirects_file = bz2.open(redirects_filename, mode='rt')
redirects_data = redirects_file.readlines()
redirects_file.close()

# pagelinks data has 119M entries
# We will only consider the first 5M for this question
pagelinks_file = bz2.open(page_links_filename, mode='rt')
pagelinks_data = [next(pagelinks_file) for x in range(5000000)] 
pagelinks_file.close()

In [4]:
# looking at the size of the data and some examples
print ('The number of entries in redirects:', len(redirects_data))
print ('A couple of examples from redirects:')
print (redirects_data[10000:10002])
print ('\n')

print ('The number of entries in pagelinks:', len(pagelinks_data))
print ('A couple of examples from pagelinks:')
print (pagelinks_data[100000:100002])

The number of entries in redirects: 4082533
A couple of examples from redirects:
['<http://dbpedia.org/resource/Proper_superset> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/Subset> .\n', '<http://dbpedia.org/resource/Jean_Paul_Sartre> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/Jean-Paul_Sartre> .\n']


The number of entries in pagelinks: 5000000
A couple of examples from pagelinks:
['<http://dbpedia.org/resource/Antipope> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Council_of_Constance> .\n', '<http://dbpedia.org/resource/Antipope> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Pope_Alexander_V> .\n']


#### <font color="red">Note:</font> It is worth noting here that each article is uniquely represented by its URL, or rather, the last segment of its URL



---



### Function `get_article_name` which takes as input the URL string, and extracts the article name from the url

In [5]:
def get_article_name(url):
    prefix = "<http://dbpedia.org/"
    prefix_index = url.find(prefix)
    p1 = url.find("/", prefix_index + len(prefix))
    p2 = url.rindex(">")
    article_name = url[p1+1:p2]
    return article_name



---


### Function `resolve_redirects` which takes as input a list of redirect lines, and returns a map between the initial and the resolved redirect page.

e.g.: input = \
[
'\<http://dbpedia.org/resource/A> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/B> .\n',\
'\<http://dbpedia.org/resource/B> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/C> .\n',\
'\<http://dbpedia.org/resource/C> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/D> .\n',\
'\<http://dbpedia.org/resource/X> \<http://dbpedia.org/property/redirect> \<http://dbpedia.org/resource/Z> .\n'
]

output = {'A': 'D', 'B': 'D', 'C': 'D', 'X': 'Z'}

#### <font color="red">Note:</font> Ignoring malformed lines which are those which do not split in 4 parts.

In [6]:
import itertools as it

def resolve_redirects(redirect_lines):
    
    resolve_redirects = {}
    for l, url in enumerate(redirect_lines):
        split_url = url.split()
        if len(split_url) != 4:
            continue
        resolve_redirects[get_article_name(split_url[0])] = get_article_name(split_url[2])

    for to, from_ in enumerate(resolve_redirects.keys()):
        tt = None
        t = resolve_redirects[from_]
        seen = {from_}
        while True:
            tt = t
            t = resolve_redirects.get(t)
            if t is None or t in seen:
                break
            seen.add(t)
        resolve_redirects[from_] = tt

    return resolve_redirects



---



### Article-article adjacency matrix. 
### Let the number of articles $n$. The adjacency matrix should have a value $A[i][j]=1$ if there is a link from $i$ to $j$. Note that the matrix may not be symmetric. This matrix would have rows as source, and columns as destinations. However, for further sections, we need it the other way round. Therefore, return $A^\top$ matrix. 
### Function `make_adjacency_matrix` that takes as input the resolved redirect map from above, and the list from `pagelinks_data`. Returns a tuple of `(index_map, A')`, where `index_map` is a map of each article to a unique number between $0$ and $n-1$, also its unique numerical id. `A` is the adjacency matrix in [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) format. `A'` is the transpose of matrix `A`. 


#### <font color="red">Note:</font> If A redirects to D and X redirects to Y, and there is a pagelink entry from A to X, then the resolved pagelink entry should be D to Y.

In [7]:
from scipy.sparse import csr_matrix

def index_of(redirects, index_map, k):
    k = redirects.get(k, k)
    return index_map.setdefault(k, len(index_map))

def make_adjacency_matrix(resolved_redirects, pagelinks_data):
    index_map = dict()
    links = list()
    for l, url in enumerate(pagelinks_data):
        split_url = url.split()
        if len(split_url) != 4:
            continue
        i = index_of(resolved_redirects, index_map, get_article_name(split_url[0]))
        j = index_of(resolved_redirects, index_map, get_article_name(split_url[2]))
        links.append((i, j))

    A = sparse.lil_matrix((len(index_map), len(index_map)), dtype=np.float32)
    for i, j in links:
        A[i, j] = 1.0

    A = A.tocsr()
    
    return index_map,A.T



---



### Applying the above functions on the dataset to create adjacency matrix $A$ and other relevant maps as directed below. Then applying `SVD` from sklearn on the adjacency matrix to determine principal singular vectors. 

In [8]:
# 1. with redirects_data as input, use the resolve_redirects function to generate the redirects_map
redirects_map = resolve_redirects(redirects_data)

# 2. with redirects map from previous step pagelinks_data as inputs, use the make_adjacency_matrix to generate index_map and adjacency_matrix
index_map, X = make_adjacency_matrix(redirects_map,pagelinks_data)

# 3. using index_map, create a reverse_index_map, which has the article name as key, and its index as value
reverse_index_map = dict((v,k) for k,v in index_map.items())

### Applying ```randomized_svd``` from sklearn on the adjacency matrix. Extracting top 5 components and running for 3 iterations.

In [9]:
from sklearn.utils.extmath import randomized_svd
U, s, V = randomized_svd(X,5,n_iter=3)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


In [10]:
# now, we print the names of the wikipedia related strongest components of the
# principal singular vector which should be similar to the highest eigenvector
print("Top wikipedia pages according to principal singular vectors")
pp.pprint([reverse_index_map[i] for i in np.abs(U.T[0]).argsort()[-10:]])
pp.pprint([reverse_index_map[i] for i in np.abs(V[0]).argsort()[-10:]])

Top wikipedia pages according to principal singular vectors
[   'England',
    'Spain',
    'Italy',
    'Canada',
    'Japan',
    'Germany',
    'World_War_II',
    'France',
    'United_Kingdom',
    'United_States']
['1989', '1971', '1975', '1970', '2006', '1972', '1996', '1966', '1967', '2007']




---



## The Pagerank Algorithm 
### Implementing the google pagerank algorithm by computing principal eigenvector using power iteration method.

### To start with the power iteration method, we first need to make the matrix $X$ obtained above *column stochastic*. A column stochastic matrix is a matrix in which each element represents a probability and the sum each column adds up to 1. Recall that $X$ is a matrix where the rows represent the destination and columns represents the source. The probability of visiting any destination from the source $s$ is $1/k$, where $k$ is the total number of outgoing links from $s$. Using this information to make the matrix column stochastic.  


In [11]:
n = X.shape[0]
X = X.copy()
incoming_nodes = np.asarray(X.sum(axis=1)).ravel()

for i in incoming_nodes.nonzero()[0]:
    X.data[X.indptr[i] : X.indptr[i + 1]] *= 1.0 / incoming_nodes[i]

### *Dangling Nodes*: There may exisit some pages which have no outgoing links. These are called as dangling nodes. If a random surfer just follows outgoing page links, then such a person can never leave a dangling node. We cannot just skip such a node, as there may be many pages pointing to this page, and could therefore be important. 
### To solve this problem, we introduce **teleportation** which says that a random surfer will visit an outgoing link with $\beta$ probability and can randomly jump to some other page with a $(1-\beta)/n$ probability (like through bookmarks, directly going through URL, etc.). Here $n$ is the total number of pages under consideration, and $\beta$ is called the damping factor. So now, the modified transition matrix is:
### $R = \beta X + \frac{(1-\beta)}{n} I_{n\times n}$

### where $X$ is the column stochastic matrix from previous step, and $I_{n\times n}$ is a $n\times n$ identity matrix.

### Using the transition matrix $R$, use the power iteration method to solve for the principal eigenvector $\mathbf{p}_{n\times 1}$. Start with an initial guess of $\mathbf{p}_{n\times 1}=[\frac{1}{n}, \frac{1}{n}, ..., \frac{1}{n}]$, which intuitively represents that a random surfer can start at any page with a $\frac{1}{n}$ probability. Use a damping factor of $0.85$, and perform a maximum of 100 iterations.

### Reporting the top 10 page names which correspond to the top 10 scores (magnitudes) in the principal eigenvector.

In [12]:
max_iter = 100
beta = 0.85
tol = 1e-10
dangle = np.asarray(np.where(np.isclose(X.sum(axis=1), 0), 1.0 / n, 0)).ravel()

scores = np.full(n, 1.0 / n, dtype=np.float32)  # initial guess vector 
for i in range(max_iter):
    prev_scores = scores
    scores = (
        beta * (scores * X + np.dot(dangle, prev_scores))
        + (1 - beta) * prev_scores.sum() / n
    )

    scores_max = np.abs(scores).max()
    if scores_max == 0.0:
        scores_max = 1.0
    err = np.abs(scores - prev_scores).max() / scores_max
    if err < n * tol:
        break

In [13]:
print("Top 10 scores and the corresponding pages")
top_indexes = np.argsort(scores)[:10]
rank=0
for k in top_indexes:
     if k in reverse_index_map:
        rank +=1
        print("Rank = ",rank,"Page name = ",reverse_index_map[k])

Top 10 scores and the corresponding pages
Rank =  1 Page name =  Glasshouse
Rank =  2 Page name =  Dark_Mansions
Rank =  3 Page name =  Happy_Birthday_to_Me
Rank =  4 Page name =  Skatetown%2C_U.S.A.
Rank =  5 Page name =  James_at_15
Rank =  6 Page name =  The_Loneliest_Runner
Rank =  7 Page name =  Little_House_on_the_Prairie_%28film%29
Rank =  8 Page name =  TP_de_Oro
Rank =  9 Page name =  Which_Mother_Is_Mine%3F
Rank =  10 Page name =  The_Egg




---

